# ReaLSAT: Reservoir and Lake Surface Area Timseries Dataset

*   This notebook acommpanies the ReaLSAT dataset available at https://zenodo.org/record/5039223

*   The notebook enables generation of surface area extents (and timeseries) of any ReaLSAT waterbody using GSW's global pixel maps[1] at monthly scale as input and the ORBIT framework[2].

*   The notebook assumes Google Colab as the running environment. The users are encouraged to adapt the code by installing required packages locally in order to use the notebook outside Google Colab.

*   The code requires authentication to use Google Earth Engine's API as well as Google Drive.

*   The code also requires the base shapefile containing ReaLSAT waterbodies to be present in a Google Drive folder.



####  

##### 1. Pekel, J. F., Cottam, A., Gorelick, N., & Belward, A. S. (2016). High-resolution mapping of global surface water and its long-term changes. Nature, 540(7633), 418-422.

##### 2.Khandelwal, A. (2019). ORBIT (Ordering Based Information Transfer): A Physics Guided Machine Learning Framework to Monitor the Dynamics of Water Bodies at a Global Scale (Doctoral dissertation, University of Minnesota).

### Import packages



In [ ]:
import ee
from google.colab import drive
!pip install geopandas
import geopandas
import json
import numpy as np
from IPython.display import clear_output
import gdal
import matplotlib.pyplot as plt
import os
import skimage.measure
from tqdm import tqdm
import time

### Authenticate APIs

In [ ]:
# Trigger the authentication for Google Earth Engine API
# The code will ask to paste the token in the input box
ee.Authenticate()

# Initialize the library.
ee.Initialize()


#mount google drive
# The code will ask to paste the token in the input box 
drive.mount('/content/gdrive')


### Load ReaLSAT waterbodies

In [ ]:
google_drive_folder = 'ReaLSAT_ORBIT' # ensure that this folder exists in your google drive
realsat_file_name = 'ReaLSAT' # name of the shapefile that contains the reference shape of all waterbodies. This shapefile should be the above google drive folder.

In [ ]:
### load ReaLSAT base shapefile
realsat = geopandas.read_file('/content/gdrive/MyDrive/' + google_drive_folder + '/' + realsat_file_name + '.shp')
realsat

### extract GSW monthly maps

In [ ]:
# use any geospatial analyis software (e.g. QGIS) to visualize waterbodies in realsat shapefile and select the ID to process.
ID = 447286 # an agricultural reservoir in Uruguay.


In [ ]:
# extract bounding box
geom = realsat[realsat['ID']==ID].buffer(0.01).iloc[0]
g = json.loads(realsat[realsat['ID']==ID].buffer(0.01).to_json())
coords = np.array(g['features'][0]['geometry']['coordinates']).tolist()
bbox = ee.Geometry.Polygon(coords)

# load GSW monthly maps
jrc_ee = ee.ImageCollection("JRC/GSW1_3/MonthlyHistory").toBands().clip(bbox)

# download the clipped monthly maps in the google drive folder
task = ee.batch.Export.image.toDrive(image=jrc_ee,
                                     description='montly maps',
                                     scale=30,
                                     region=bbox,
                                     fileNamePrefix='ID_' + str(ID).zfill(6),
                                     folder= google_drive_folder,
                                     crs='EPSG:4326',
                                     maxPixels= 1e13,
                                     fileFormat='GeoTIFF')
task.start()
while(1):
  print('task status: ' + task.status()['state'])
  if task.status()['state']=='COMPLETED':
    break
  time.sleep(10)
  clear_output(wait=True)

#### Preprocess monthly maps

In [ ]:
# utility function to convert a python n-d array to a geotiff file
def create_geo_tiff(data,outfile,basefile,odtype):
    rasterFormat = 'GTiff' # for now assuming output format is going to GTiff
    rasterDriver = gdal.GetDriverByName(rasterFormat)
    ds = gdal.Open(basefile)
    geotransform = ds.GetGeoTransform()
    projection = ds.GetProjection()


    band = ds.GetRasterBand(1)
    full_xsize = band.XSize
    full_ysize = band.YSize
    ds = None
    mds = rasterDriver.Create(outfile,full_xsize,full_ysize,data.shape[2],odtype)
    mds.SetGeoTransform(geotransform)
    mds.SetProjection(projection)
    # initializing data array and putting zero filled bands in the prediction raster
    for i in range(data.shape[2]):
        mds.GetRasterBand(i+1).WriteArray(data[:,:,i])
    # closing datasets
    mds = None
    return

In [ ]:
# load monthly maps
input_jrc = gdal.Open('/content/gdrive/MyDrive/' + google_drive_folder + '/ID_' + str(ID).zfill(6) + '.tif',0).ReadAsArray()
T,row,col = input_jrc.shape

# reshuffle pixel labels, before reshuffle land is 1, water is 2, and unknown is 0
input_jrc[input_jrc==0] = 3
input_jrc[input_jrc==1] = 0
input_jrc[input_jrc==2] = 1
input_jrc[input_jrc==3] = 2
# after reshuffle, land is 0, water is 1, and unknown is 2

# calculate % timesteps where a pixel is water
occurrence = np.sum(input_jrc==1,axis=0)*1.0/np.sum(input_jrc<=1,axis=0)

# prepare an empty tif file to map the selected waterbody's reference shape on pixels in the array
base_data = np.expand_dims(input_jrc[0]*0.0,axis=2)
create_geo_tiff(base_data,'/content/gdrive/MyDrive/' + google_drive_folder + '/ID_' + str(ID).zfill(6) + '_ref.tif','/content/gdrive/MyDrive/' + google_drive_folder + '/ID_' + str(ID).zfill(6) + '.tif',gdal.GDT_Int32)
os.system('gdal_rasterize -a ID -l ' + realsat_file_name + ' ' + '/content/gdrive/MyDrive/' + google_drive_folder + '/' + realsat_file_name + '.shp ' + '/content/gdrive/MyDrive/' + google_drive_folder + '/ID_' + str(ID).zfill(6) + '_ref.tif')

# load the reference shape mask
mask = gdal.Open('/content/gdrive/MyDrive/' + google_drive_folder + '/ID_' + str(ID).zfill(6) + '_ref.tif',0).ReadAsArray()==ID

# identify pixels that are not part of the selected lake
# array to count number of timesteps a pixel is not a part of a connected component
# that overlaps with the reference shape
unconnected_count = np.zeros((row,col))
# looping through each timestep
for t in tqdm(range(T)):
  cur_extent_map = input_jrc[t]
  cur_conn_comps = skimage.measure.label(cur_extent_map==1)
  ulabels = np.unique(cur_conn_comps)
  # if the map is completely dry, skip
  if len(ulabels)==1:
    continue
  ulabels = ulabels[1:] # remove background class (0)
  for ulabel in ulabels:
    cur_comp = cur_conn_comps==ulabel
    if np.sum(cur_comp[mask])==0: # zero overlap with reference shape
      unconnected_count[cur_comp] =  unconnected_count[cur_comp] + 1


# remove pixels that are unconnected 
exclude_map = np.logical_or(np.isnan(occurrence),np.logical_and(occurrence>0,unconnected_count>5))

# identify pixels that are always water or land to avoid unneccessary computations for ORBIT based label correction
permanent_water = np.logical_and(occurrence>0.99,exclude_map==0)
permanent_land = np.logical_and(occurrence<0.01,exclude_map==0)

# identify dynamic pixel
dynamic_pixels = np.logical_and(np.logical_and(exclude_map==0,permanent_water==0),permanent_land==0)
dynamic_indices = np.where(dynamic_pixels.flatten()==1)[0]

D  = np.sum(dynamic_pixels==1)

# convert flatten each timestep to prepare array for ORBIT based correction workflow.
input_stack = np.zeros((D,T))
for t in tqdm(range(T)):
  temp = input_jrc[t].flatten()
  input_stack[:,t] = temp[dynamic_indices]


# visualize different pixel categories
f,ax = plt.subplots(1,6,figsize=(15,5))
ax[0].imshow(occurrence)
ax[0].set_title('Occurrence Map')
ax[1].imshow(mask)
ax[1].set_title('Reference Shape')
ax[2].imshow(permanent_water)
ax[2].set_title('Permanent Water')
ax[3].imshow(permanent_land)
ax[3].set_title('Permanent Land')
ax[4].imshow(dynamic_pixels)
ax[4].set_title('Dynamic Pixels')
ax[5].imshow(exclude_map)
ax[5].set_title('Excluded Pixels')
plt.tight_layout()
plt.show()  

#### ORBIT based label correction and imputation

In [ ]:
# Procedure to correct and impute labels to most likely labels.
# Please see the following citations for more details: 

def ORBIT_S(data):
  N,T = data.shape
  cor_data = np.zeros((data.shape))
  for t in range(T):
    ord_labels = data[:,t]
    ord_labels = np.append(0,ord_labels)
    sum1 = np.cumsum(ord_labels)
    sum2 = np.cumsum(1 - ord_labels)
    score = sum1 + np.sum(sum2) - sum2
    ind = np.argmin(score)

    new_labels = ord_labels.copy()
    new_labels[:] = 0
    new_labels[0:ind] = 0
    new_labels[ind:] = 1
    new_labels = new_labels[1:]
    cor_data[:,t] = new_labels
  return cor_data
    

# learns a relative elevation ordering using multi-temporal extent maps
def get_relative_elvation_ordering(data,max_iterations=10):
  
  data[data==2] = 0.5 # replace missing pixels with 0.5 to represent equal chance of being land or water
  N,T = data.shape
  dix = np.random.permutation(N) # initialize ordering
  ord_data = data[dix,:]
  wcount = np.sum(ord_data==1,axis=1)*1.0/np.sum(ord_data<=1,axis=1)
  est_err = []
  for inum in range(max_iterations):
    cor_data = ORBIT_S(ord_data.copy()) # correct and impute pixels based on current ordering
    
    # calculate cost of current ordering. Low cost means less changes to input labels to achieve physical consistency
    ers = np.sum(np.abs(cor_data-ord_data)) 
    est_err.append(ers)
    print('Loss at iteration {}: {}'.format(inum,ers))
    # stop at convergence
    if len(est_err)>1 and est_err[-1]>=est_err[-2]:
        break

    #calculate new ordering 
    Mmat = np.zeros((N,))
    for i in tqdm(range(N)):
        cur_label = np.expand_dims(ord_data[i,:],axis=0)
        cur_label = np.repeat(cur_label,N,axis=0)
        diff_label = np.abs(cor_data-cur_label)
        diff_count = np.sum(diff_label,axis=1)
        min_inds = diff_count==np.min(diff_count)
        
        # breaking across-profile tie by picking the elevation profile with 
        # largest rank in the current iteration
        last_ind = np.where(min_inds==1)[0][-1]

        # breaking tie randomly
        # last_ind = np.random.permutation(np.where(min_inds==1)[0])[0]
        
        Mmat[i] = last_ind
    
    tinfo = [(Mmat[i],wcount[i]) for i in range(N)]
    tinfo = np.array(tinfo,dtype=np.dtype([('x', float), ('y', float)]))
    fix = np.argsort(tinfo)
    # print(fix.shape)
    ord_data = ord_data[fix,:]
    dix = dix[fix]
    wcount = np.sum(ord_data==1,axis=1)*1.0/np.sum(ord_data<=1,axis=1)

  return dix
    

# use dynamic programming to generate temporally smooth extent maps
def ORBIT_T(local_score,cth):

    row,col = local_score.shape
    best_score = np.zeros((row,col))
    best_score[:,0] = local_score[:,0]
    best_index = np.zeros((row,col))
    row_base = np.arange(row)
    for j in tqdm(range(1,col)):
        for i in range(row):
            trn_cost = np.abs(i-row_base)*1.0*cth
            lcl_cost = local_score[i,j]
            prv_cost = best_score[:,j-1]
            ttl_cost = trn_cost + lcl_cost + prv_cost
            bst_cost = np.min(ttl_cost)
            bst_indx = np.argmin(ttl_cost)
            best_score[i,j] = bst_cost
            best_index[i,j] = bst_indx

    cuts = np.zeros((col,))
    bst_cst = best_score[:,-1]
    bst_ind = int(np.argmin(bst_cst))

    for j in range(col-1,-1,-1):
        cuts[j] = bst_ind
        bst_ind = int(best_index[bst_ind,j])

    return cuts



In [ ]:
ix_orbit = get_relative_elvation_ordering(input_stack.copy(),max_iterations=10)

ord_labels = np.zeros((D+1,T))
ord_labels[1:,:] = input_stack[ix_orbit,:]
weight = 3.0 # penality on changing water pixels compared to land pixels
thr = 0.3 # smoothness parameter: higher value would lead to more smoothing
sum1 = np.cumsum(ord_labels==1,axis=0)*weight
sum2 = np.cumsum(ord_labels==0,axis=0)
score = sum1 + np.sum(sum2) - sum2
print('running ORBIT-T')
cuts = ORBIT_T(score.copy(),thr)


In [ ]:
# array to store updated extent maps
orbit_stack = np.zeros((row,col,T)).astype(np.uint8)

jrc_timeseries = np.zeros((T,))
orbit_timeseries = np.zeros((T,))
for t in tqdm(range(T)):
    cur_cut = int(cuts[t])
    cur_map = np.zeros((D,))
    cur_map[cur_cut:] = 1
    cur_map_full = np.zeros((row*col,))
    cur_map_full[dynamic_indices[ix_orbit]] = cur_map
    cur_map_full = np.reshape(cur_map_full.copy(),(row,col))
    cur_map_full[permanent_land] = 0
    cur_map_full[permanent_water] = 1
    orbit_stack[:,:,t] = cur_map_full
    jrc_timeseries[t] = np.sum(input_jrc[t][exclude_map==0]==1)
    orbit_timeseries[t]= np.sum(cur_map_full[exclude_map==0]==1)


In [ ]:
# compare GSW based timeseries with ORBIT based timeseries
plt.figure(figsize=(15,5))
plt.plot(jrc_timeseries,'.-r',label='JRC Area')
plt.plot(orbit_timeseries,'.-b',label='ORBIT Area')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# extract date information
date_list = []
temp = jrc_ee.getInfo()
for t in tqdm(range(T)):
  date_list.append(temp['bands'][t]['id'][0:7])

# visualize and compare surface extent maps
for t in range(300,T):
  print(t,date_list[t])
  fig = plt.figure(figsize=(20, 5))
  ax1= fig.add_subplot(1,4,1)
  ax2= fig.add_subplot(1,4,2)
  ax3= fig.add_subplot(1,2,2)

  cur_jrc = input_jrc[t]
  ax1.imshow(input_jrc[t],vmin=0,vmax=2)
  ax2.imshow(orbit_stack[:,:,t],vmin=0,vmax=2)
  ax1.set_title('JRC Map')
  ax2.set_title('ORBIT Map')
  ax3.plot(jrc_timeseries,'.-r',label='JRC Area')
  ax3.plot(orbit_timeseries,'.-b',label='ORBIT Area')
  ax3.plot([t,t],[0,np.max(orbit_timeseries)],'--k')
  ax3.set_title(date_list[t])
  plt.tight_layout()
  plt.show()
  time.sleep(1)
  clear_output(wait=True)



In [ ]:
#reshuffle pixel labels to match GSW categories
orbit_stack[orbit_stack==0] = 3
orbit_stack[orbit_stack==2] = 0
orbit_stack[orbit_stack==1] = 2
orbit_stack[orbit_stack==3] = 1

# save updated labels
create_geo_tiff(orbit_stack,'/content/gdrive/MyDrive/' + google_drive_folder + '/ID_' + str(ID).zfill(6) + '_orbit_updated.tif','/content/gdrive/MyDrive/' + google_drive_folder + '/ID_' + str(ID).zfill(6) + '.tif',gdal.GDT_Byte)